## Generate AX25 frame

In [1]:
import ax25

callsign = b'UT3USW'
Digi =b'WIDE1-1,WIDE2-2'
dest = b"APFIIO"
info = b"/140623h5031.21N/03030.46Ea/A=000390!w?k!"

packet = ax25.UI(
            destination=dest,
            source=callsign, 
            info=info,
            digipeaters=Digi.split(b','),
        )

### Our position AX25 packet

In [2]:
print(packet.unparse())

bitarray('01111110010000010000010100110001010010010100100101111001000001100101010100010101011001100101010101100101011101010000011001110101010010010001000101010001010001100000001001000110011101010100100100010001010100010010011000000010101001101100000000001111101110100100011000010110000001100011011000100110011001100000101101010110000001100110011001000110001110100010011001000110001110010111101000000110011001100000011001100110000001100011101000010110001101100101000101000011011110100100000101011110000001100000011000000110011001100100111000000110010000100111011101111101001101011010000100110110101111010001111110')


In [3]:
print(packet.unparseToBytes())

b'~\x82\xa0\x8c\x92\x92\x9e`\xaa\xa8f\xaa\xa6\xae`\xae\x92\x88\x8ab@b\xae\x92\x88\x8ad@e\x03\xf0]bh`ldf\xd0j`fb\\db\x9c^`f`f`\\hl\x8a\xc2^\x82z```fr`B\xee\xbe\xac\x85l\xbd\xf8\x01'


In [4]:
import math

MARK_HZ = 1200.0
SPACE_HZ = 2200.0
BAUD_RATE = 1200.0

FRAME_RATE=44100

TWO_PI = 2.0 * math.pi

In [5]:
import itertools

from bitarray import bitarray

def encode(binary_data):
    framed_data = frame(binary_data)

    return multiply(modulate(framed_data), constant(0.5))

In [6]:
def nrzi(data):
    current = True
    for bit in data:
        if not bit:
            current = not current 
        yield current

In [7]:
def frame(stuffed_data):
    return nrzi(
        stuffed_data
    )

In [8]:
def modulate(data):
    seconds_per_sample = 1.0 / FRAME_RATE
    phase, seconds, bits = 0, 0, 0

    clock = (x / BAUD_RATE for x in itertools.count(1))
    tones = (MARK_HZ if bit else SPACE_HZ for bit in data)

    for boundary, frequency in zip(clock, tones):
        phase_change_per_sample = TWO_PI / (FRAME_RATE / frequency)
        while seconds < boundary:
            yield math.sin(phase)

            seconds += seconds_per_sample
            phase += phase_change_per_sample

            if phase > TWO_PI:
                phase -= TWO_PI

        bits += 1

In [9]:
def silence(seconds=None):
    if seconds != None:
        for i in range(int(FRAME_RATE * seconds)):
            yield 0
    else:
        while True:
            yield 0

In [10]:
def encode(binary_data):
    framed_data = frame(binary_data)

    for sample in itertools.chain(
        silence(1.05), 
        modulate(framed_data), 
        silence(1.05), 
    ):
        yield sample

In [11]:
import wave
import struct

num_channels = 1
sampwidth = 2

with wave.open('output.wav', 'w') as wav_file:
    wav_file.setnchannels(num_channels)
    wav_file.setsampwidth(sampwidth)
    wav_file.setframerate(FRAME_RATE)

    bitarray_fr = packet.unparse()

    str1 = "";
    for b in bitarray_fr:
        str1 += str(b);

    new = bitarray(str1)

    for sample in encode(bitarray_fr):
        scaled_sample = int(max(min(sample * 32767.0, 32767), -32768))
        wav_file.writeframes(scaled_sample.to_bytes(2, byteorder='little', signed=True) )
    print ("DONE")

DONE
